In [2]:
# Kaggle Install 
#! pip install -Uq kaggle
#! pip install -Uq fastai==2.2.5

You should consider upgrading via the 'c:\users\jeron\anaconda3\python.exe -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement torchvision<0.9,>=0.8 (from fastai) (from versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.5.0, 0.9.0, 0.9.1, 0.10.0, 0.10.1, 0.11.0, 0.11.1)
ERROR: No matching distribution found for torchvision<0.9,>=0.8
You should consider upgrading via the 'c:\users\jeron\anaconda3\python.exe -m pip install --upgrade pip' command.


In [16]:
# Kaggle Directory 
#! mkdir "C:\Users\jeron\9445 Finals Project Team U and X\.kaggle"
! copy "C:\Users\jeron\9445 Finals Project Team U and X\.kaggle\kaggle.json"
# chmod 600 C:\Users\jeron\9445 Finals Project Team U and X\.kaggle\kaggle.json --> Download "Git for Windows" and run command through there

        1 file(s) copied.


In [19]:
# Make sure to copy the kaggle.json to Users/""/.kaggle folder; make .kaggle folder if not available
#! kaggle datasets download nikitarom/planets-dataset
# Unzip file manually once downloaded, file will be 1.5 gb so download might take a while

'unzip' is not recognized as an internal or external command,
operable program or batch file.


ModuleNotFoundError: No module named 'google.colab'